<a href="https://colab.research.google.com/github/GaoangLiu/AA_ipynb/blob/master/Titanic_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
!curl -o t.zip ali.140714.xyz:8000/titanic.zip
!unzip -o t.zip 
!mkdir subs
!ls 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35066  100 35066    0     0  82702      0 --:--:-- --:--:-- --:--:-- 82508
Archive:  t.zip
  inflating: data/labels.csv         
  inflating: data/test.csv           
  inflating: data/train.csv          
  inflating: data/gender_submission.csv  
mkdir: cannot create directory ‘subs’: File exists
classboost.py  data  __pycache__  sample_data  subs  t.zip


In [0]:
import pandas as pd
import numpy as np
# pd.set_option("display.max_columns", 9999)

def birdview(data):
    print("----------Head 5 Record----------")
    print(data.head(5))
    print("\n-----------Information-----------")
    print(data.info())
    print("\n-----------Data Types-----------")
    print(data.dtypes)
    print("\n----------Missing value-----------")
    print(data.isnull().sum())
    print("\n----------Null value-----------")
    print(data.isna().sum())
    print("\n----------Shape of Data----------")
    print(data.shape)
    
def graph_insight(data):
    print(set(data.dtypes.tolist()))
    df_num = data.select_dtypes(include = ['float64', 'int64'])
    df_num.hist(figsize=(16, 16), bins=50, xlabelsize=8, ylabelsize=8);

def distribution_insight(data, column):
    print("===================================")
    print("Min Value:", data[column].min())
    print("Max Value:", data[column].max())
    print("Average Value:", data[column].mean())
    print("Center Point of Data (median):", data[column].median())
    print("===================================")
    # sns.boxplot(data[column])
    

In [117]:
train = pd.read_csv('data/train.csv')
labels = train['Survived']

test = pd.read_csv('data/test.csv')
# birdview(test)
all = pd.concat([train.reset_index(drop=True), test.reset_index(drop=True)], axis=0)
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [118]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [119]:
train = all.copy()
df0 = train.copy()
df0.drop(columns=['Survived', 'PassengerId', 'Cabin'], inplace=True)
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


## Encode sex and Embarked

In [120]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
df1 = df0.copy()
lb = LabelBinarizer()
sex = lb.fit_transform(train['Sex'])
df1['Sex'] = sex

"""
Fill the null value of Embarked
train[train.Pclass == 1].Embarked.value_counts() shows the most frequent Embarked place is S
train[train.Embarked.isna()]
"""
df1['Embarked'].fillna('S', inplace=True)
lb = LabelBinarizer()
_embarked = lb.fit_transform(df1['Embarked'])
lb_embarked = pd.DataFrame(_embarked, columns=('Embarked_' + c for c in lb.classes_))

df2 = pd.concat([df1.reset_index(drop=True), lb_embarked.reset_index(drop=True)], axis=1)
df2

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Embarked_C,Embarked_Q,Embarked_S
0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,S,0,0,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C,1,0,0
2,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,S,0,0,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,S,0,0,1
4,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,S,0,0,1
1305,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C,1,0,0
1306,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,0,0,1
1307,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,S,0,0,1


## Fill the age
For the age column, 177 out of 891 were missing, we're going to adopt the following strategy:
* replacing the missing age with mean value of passengers from the same class with the same sex

In [121]:
""" 
1. fill NaN age with mode of passengers from the same class and having same gender
2. categorize age by dividing each to 10
"""
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

nums = df2[['Age', 'Fare']]
imputer = IterativeImputer(random_state=0, max_iter=100)
nums = pd.DataFrame(imputer.fit_transform(nums), columns=nums.columns)

df3 = df2.copy()
df3['Age'] = nums.Age // 10
df3['Fare'] = nums.Fare
df3

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Embarked_C,Embarked_Q,Embarked_S
0,3,"Braund, Mr. Owen Harris",1,2.0,1,0,A/5 21171,7.2500,S,0,0,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,3.0,1,0,PC 17599,71.2833,C,1,0,0
2,3,"Heikkinen, Miss. Laina",0,2.0,0,0,STON/O2. 3101282,7.9250,S,0,0,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,3.0,1,0,113803,53.1000,S,0,0,1
4,3,"Allen, Mr. William Henry",1,3.0,0,0,373450,8.0500,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",1,2.0,0,0,A.5. 3236,8.0500,S,0,0,1
1305,1,"Oliva y Ocana, Dona. Fermina",0,3.0,0,0,PC 17758,108.9000,C,1,0,0
1306,3,"Saether, Mr. Simon Sivertsen",1,3.0,0,0,SOTON/O.Q. 3101262,7.2500,S,0,0,1
1307,3,"Ware, Mr. Frederick",1,2.0,0,0,359309,8.0500,S,0,0,1


In [122]:
"""
1. Extra titles from names. E.g., Mr. Ms.
2. Label binarizer them into multiple classes
"""
import re

df4 = df3.copy()
df4['title'] = df3.Name.map(lambda n: re.compile("([A-Za-z]+)\.").search(n).group())

df4['title'] = df4['title'].replace(['Lady.', 'Capt.', 'Col.', 'Don.', 'Dr.', 'Major.', 'Rev.', 'Jonkheer.', 'Dona.'], 'Rare.')
df4['title'] = df4['title'].replace(['Countess.', 'Lady', 'Sir'], 'Royal')
df4['title'] = df4['title'].replace(['Mlle.', 'Ms.'], 'Miss.')
df4['title'] = df4['title'].replace('Mme.', 'Mrs.')
df4['title'] = df4['title'].replace('Sir.', 'Mr.')

lb = LabelBinarizer()
vecs = lb.fit_transform(df4.title)
title_1 = pd.DataFrame(vecs, columns=lb.classes_)

df5 = pd.concat([df4.reset_index(drop=True), title_1.reset_index(drop=True)], axis=1)
df5.drop(columns=['Name', 'title', 'Ticket', 'Embarked'], inplace=True)
df5

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Master.,Miss.,Mr.,Mrs.,Rare.,Royal
0,3,1,2.0,1,0,7.2500,0,0,1,0,0,1,0,0,0
1,1,0,3.0,1,0,71.2833,1,0,0,0,0,0,1,0,0
2,3,0,2.0,0,0,7.9250,0,0,1,0,1,0,0,0,0
3,1,0,3.0,1,0,53.1000,0,0,1,0,0,0,1,0,0
4,3,1,3.0,0,0,8.0500,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0
1305,1,0,3.0,0,0,108.9000,1,0,0,0,0,0,0,1,0
1306,3,1,3.0,0,0,7.2500,0,0,1,0,0,1,0,0,0
1307,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0


In [123]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler

df6 = df5.copy()
# df6.loc[df6.Fare >= 270, 'Fare'] = 270
# df6['Fare'].fillna(df6['Fare'].mode()[0], inplace = True)
df6


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Master.,Miss.,Mr.,Mrs.,Rare.,Royal
0,3,1,2.0,1,0,7.2500,0,0,1,0,0,1,0,0,0
1,1,0,3.0,1,0,71.2833,1,0,0,0,0,0,1,0,0
2,3,0,2.0,0,0,7.9250,0,0,1,0,1,0,0,0,0
3,1,0,3.0,1,0,53.1000,0,0,1,0,0,0,1,0,0
4,3,1,3.0,0,0,8.0500,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0
1305,1,0,3.0,0,0,108.9000,1,0,0,0,0,0,0,1,0
1306,3,1,3.0,0,0,7.2500,0,0,1,0,0,1,0,0,0
1307,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0


In [124]:
"""
Feature engeneering
1. extra family size from sibsp and parch
2. add alone feature
"""
df7 = df6.copy()
df7['family_size'] = 1 + df7['SibSp'] + df7['Parch']
df7['alone'] = df7['family_size'].apply(lambda x: 1 if x == 1 else 0)
df7

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Master.,Miss.,Mr.,Mrs.,Rare.,Royal,family_size,alone
0,3,1,2.0,1,0,7.2500,0,0,1,0,0,1,0,0,0,2,0
1,1,0,3.0,1,0,71.2833,1,0,0,0,0,0,1,0,0,2,0
2,3,0,2.0,0,0,7.9250,0,0,1,0,1,0,0,0,0,1,1
3,1,0,3.0,1,0,53.1000,0,0,1,0,0,0,1,0,0,2,0
4,3,1,3.0,0,0,8.0500,0,0,1,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0,1,1
1305,1,0,3.0,0,0,108.9000,1,0,0,0,0,0,0,1,0,1,1
1306,3,1,3.0,0,0,7.2500,0,0,1,0,0,1,0,0,0,1,1
1307,3,1,2.0,0,0,8.0500,0,0,1,0,0,1,0,0,0,1,1


# Build model and train

In [0]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score

train_df = df7.loc[:890, :]
test_df = df7.loc[891:, :]
X_train, X_val, y_train, y_val = train_test_split(train_df, labels, stratify=labels, random_state=0)

In [0]:
"""
Different model might be used, thus we name submissions by
1. score
2. model name
"""
def make_predictions(model, modelname='xgboost'):
    preds = model.predict(test_df)
    y_real = pd.read_csv('data/labels.csv').Survived
    print("Test accuracy score", accuracy_score(y_real, preds))

    subs = pd.read_csv('data/gender_submission.csv')
    subs['Survived'] = preds
    export_name = str(accuracy_score(y_real, preds))[:7] + f"_{modelname}.csv"
    subs.to_csv(f"subs/{export_name}", index=False)


In [127]:
"""
Try xgboost classifier
"""
import xgboost
from sklearn.metrics import accuracy_score

model = xgboost.XGBClassifier()
model.fit(X_train, y_train)
y_preds = model.predict(X_val)
print("Accuracy score", accuracy_score(y_val, y_preds))
make_predictions(model)

Accuracy score 0.8116591928251121
Test accuracy score 0.7631578947368421


In [128]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_preds = lr.predict(X_val)
print("Accuracy score", accuracy_score(y_val, y_preds))
make_predictions(lr, 'logreg')

Accuracy score 0.8161434977578476
Test accuracy score 0.7727272727272727


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [129]:
"""Try random forest
"""
from sklearn.ensemble import RandomForestClassifier

parameters = {'bootstrap': False, 'min_samples_leaf': 3, 'n_estimators': 50, 
                  'min_samples_split': 10, 'max_features': 'sqrt', 'max_depth': 6}
    
model = RandomForestClassifier(**parameters)
model.fit(X_train, y_train)
y_preds = model.predict(X_val)
print("Accuracy score", accuracy_score(y_val, y_preds))
make_predictions(model, 'random_forest')

Accuracy score 0.8251121076233184
Test accuracy score 0.7703349282296651


In [130]:
labels

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [187]:
# Load packages
import math
import re
import os
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import logging
import time
import smart_open

# TF modules
import tensorflow as tf

# sklearn modules
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import xgboost
from lightgbm import LGBMClassifier

logging.basicConfig(
	format='[%(asctime)s %(levelname)8s] %(message)s',
	level=logging.INFO,
	datefmt='%m-%d %H:%M:%S')


class Helper():
	'''For exporting, transferring files ...
	'''
	def export_prediction(
			self,
			y_preds,
			target='target',
			sample='sample_submission.csv',
			subfile='submission.csv'):
		sub = pd.read_csv(sample)
		sub[target] = y_preds
		sub.to_csv(subfile, index=False)
		logging.info(f"Exported to {subfile}")


class Classifier(object):
	""" Base class for classifier
	:INPUTS:

	:train_df: train pandas dataframe
	:test_df: test pandas dataframe
	:target: target column name (str)
	:n_splits: K in KFold (default is 4)
	:verbose: bool

	:EXAMPLE:
	"""

	def __init__(
			self,
			train_df=[],
			test_df=[],
			labels=[],
			metric='accuracy_score',
			target='target',
			verbose=True):
		self.train_df = train_df
		self.test_df = test_df
		self.labels = labels
		self.model = None
		self.metric = metric
		self.target = target

	def train_model(self):
		'''
		Choose your own classifier: self.model and run it
		'''
		logging.info("===========================================")
		logging.info(f"{self.__class__.__name__} starts training.")
		X_train, X_val, y_train, y_val = train_test_split(
			self.train_df, self.labels, stratify=self.labels, test_size=0.2, random_state=0)
		self.model.fit(X_train, y_train)

		logging.info(f"{self.__class__.__name__} starts validating on X_val")
		y_preds = self.model.predict(X_val)

		self.score = self.display_metric(y_val, y_preds)
		return self.model

	def display_metric(self, y_val, y_preds):
		'''
		Evaluate the model performance on X_val dataset and display the score
		'''
		score = -1
		if self.metric == 'accuracy_score' or self.metric == 'acc':
			score = accuracy_score(y_val, y_preds)
			logging.info(f"Accuracy score: {score}")
		elif self.metric == 'auc':
			score = roc_auc_score(y_val, y_preds)
			logging.info(f"Area under the ROC curve: {score}")
		else:
			logging.error("Your metrics {self.metric} is not supported.")
		return score

	def make_prediction(self, eager=0, extra_name_info='c'):
		'''Make predictions on test_df and export result to local file
		@parameters:
		1. eager
				1. eager == 0, do nothing
				2. eager == 1, make predictions and return, but do NOT export predictions
				3. eager == 2 +, make prediction, export predictions and return
		'''
		if eager == 0:
			logging.info("Prediction procedure aborted (eager = 0)")
			return
		y_preds = self.model.predict(self.test_df)
		if eager > 1:
			sub = pd.read_csv(f"sample_submission.csv")
			sub[self.target] = y_preds
			output = f"submission_{self.__class__.__name__}_{extra_name_info}.csv"
			sub.to_csv(output, index=False)
			logging.info(f'Prediction exported to {output}')
		return y_preds

	def pipeline(self, eager=0, extra_name_info="c"):
		'''pipeline running
		'''
		s_time = time.clock()
		self.build_model()
		self.train_model()
		y_preds = self.make_prediction(eager, extra_name_info)
		logging.info(f"Program running time: {time.clock() - s_time} seconds")
		return self.model, y_preds


class C_bayes(Classifier):
	def build_model(self):
		self.model = MultinomialNB()
		return self.model

class C_gbc(Classifier):
	def build_model(self):
		self.model = GradientBoostingClassifier(random_state=0)
		return self.model


class C_rf(Classifier):
	def build_model(self):
		parameters = {
			'bootstrap': False,
			'min_samples_leaf': 3,
			'n_estimators': 200,
			'min_samples_split': 10,
			'max_features': 'sqrt',
			'max_depth': 6,
			'n_jobs': -1}
		self.model = RandomForestClassifier(**parameters)
		return self.model


class C_xgboost(Classifier):
	def build_model(self):
		parameters = {'verbosity': 0,
					  'learning_rate': 0.05,
					  'sumsampling': 1,
					  'colsample_bytree': 1,  # number of columns used by each tree
					  'max_depth': 10,
					  'gamma': 1,
					  'n_estimators': 3000
					  }
		self.model = xgboost.XGBClassifier(**parameters)
		return self.model

	def train_model(self):
		''' xgboost classifier training relies on eval set, and we can also
		utilize earlystopping to get better performance
		'''
		logging.info(f"{self.__class__.__name__} starts training.")
		X_train, X_val, y_train, y_val = train_test_split(
			self.train_df, self.labels, stratify=self.labels, test_size=0.2, random_state=0)
		eval_set = [(X_train, y_train), (X_val, y_val)]
		eval_metric = [self.metric]

		self.model.fit(
			X_train,
			y_train,
			eval_metric=['logloss'],
			eval_set=eval_set,
			early_stopping_rounds=50,
			verbose=False)

		logging.info(f"{self.__class__.__name__} starts validating on X_val")
		y_preds = self.model.predict(X_val)

		self.score = self.display_metric(y_val, y_preds)

		return self.model


class C_lr(Classifier):
	'''Logistic Regression
	'''
	def build_model(self):
		self.model = LogisticRegression(max_iter=1000)
		return self.model
	
class C_lightgbm(Classifier):
	'''Logistic Regression
	'''
	def build_model(self):
		self.model = LGBMClassifier()
		return self.model


boost = C_xgboost(train_df=train_df, test_df=test_df, labels=labels, target='Survived')
lr = C_lr(train_df=train_df, test_df=test_df, labels=labels, target='Survived')
rf = C_rf(train_df=train_df, test_df=test_df, labels=labels, target='Survived')
gbc = C_gbc(train_df=train_df, test_df=test_df, labels=labels, target='Survived')
lgbm = C_lightgbm(train_df=train_df, test_df=test_df, labels=labels, target='Survived')

y_preds = np.zeros(test_df.shape[0])
for m in [boost, lr, rf]:
    _, curpreds = m.pipeline(eager=1)
    y_preds += curpreds 

pred1 = (y_preds >= 2).astype(int)
Helper().export_prediction(pred1, target='Survived', sample='data/gender_submission.csv', subfile='titanic_ensemble.csv')
y_preds

[06-04 06:05:46     INFO] C_xgboost starts training.
[06-04 06:05:46     INFO] C_xgboost starts validating on X_val
[06-04 06:05:46     INFO] Accuracy score: 0.8491620111731844
[06-04 06:05:46     INFO] Program running time: 0.2823990000000123 seconds
[06-04 06:05:46     INFO] ===========================================
[06-04 06:05:46     INFO] C_lr starts training.
[06-04 06:05:46     INFO] C_lr starts validating on X_val
[06-04 06:05:46     INFO] Accuracy score: 0.8100558659217877
[06-04 06:05:46     INFO] Program running time: 0.23582700000000045 seconds
[06-04 06:05:46     INFO] ===========================================
[06-04 06:05:46     INFO] C_rf starts training.
[06-04 06:05:47     INFO] C_rf starts validating on X_val
[06-04 06:05:47     INFO] Accuracy score: 0.8212290502793296
[06-04 06:05:47     INFO] Program running time: 0.7014650000000131 seconds
[06-04 06:05:47     INFO] Exported to titanic_ensemble.csv


array([0., 2., 0., 0., 3., 0., 3., 0., 3., 0., 0., 0., 3., 0., 3., 3., 0.,
       0., 2., 3., 1., 3., 3., 2., 3., 0., 3., 0., 0., 0., 0., 0., 2., 2.,
       2., 0., 2., 2., 0., 1., 0., 1., 0., 3., 3., 0., 0., 0., 3., 3., 1.,
       0., 3., 3., 0., 0., 0., 0., 0., 3., 0., 0., 0., 3., 3., 3., 3., 0.,
       0., 3., 3., 0., 2., 0., 3., 1., 0., 3., 0., 3., 3., 0., 0., 0., 0.,
       0., 3., 2., 3., 3., 2., 0., 3., 0., 0., 0., 3., 0., 3., 0., 3., 0.,
       0., 0., 2., 0., 0., 0., 0., 0., 0., 3., 3., 3., 3., 0., 0., 3., 0.,
       3., 3., 0., 3., 0., 0., 3., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 2., 0., 0., 3., 0., 0., 1., 0., 0., 0., 1., 0., 3., 0., 1.,
       3., 0., 0., 3., 3., 1., 2., 3., 3., 3., 0., 0., 3., 0., 0., 3., 2.,
       0., 0., 0., 0., 0., 3., 3., 1., 3., 3., 0., 0., 3., 0., 3., 0., 3.,
       0., 0., 0., 0., 0., 3., 0., 3., 0., 3., 3., 0., 2., 3., 3., 1., 3.,
       0., 0., 3., 0., 3., 0., 0., 0., 0., 3., 0., 0., 3., 0., 3., 0., 3.,
       0., 3., 0., 3., 2.

[06-04 05:50:48     INFO] Exported to titanic_logreg_gridcv.csv


In [188]:
!curl -X PUT --upload-file titanic_ensemble.csv ali.140714.xyz:8000/
# !cp data/gender_submission.csv sample_submission.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  2839    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

## Received: "titanic_ensemble.csv"

100  2878    0    39  100  2839     21   1597  0:00:01  0:00:01 --:--:--  1619
